**iWildCam 2019 EDA
**

The aim of this competition is to classify animal species based on images collected. 
The competition is using training data and test data from different regions, mainly Southwest America  and Northwest America. The target variable is "category_id"

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import gc
import os
import json
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt
import keras
from keras import layers
from keras.applications import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
warnings.filterwarnings('ignore')

In [ ]:
train_df=pd.read_csv('../input/train.csv')

In [ ]:
test_df=pd.read_csv('../input/test.csv')

Selecting one photo randomly for each category:

In [ ]:
fig = plt.figure(figsize=(25, 60))
imgs = [np.random.choice(train_df.loc[train_df['category_id'] == i, 'file_name'], 4) for i in train_df.category_id.unique()]
imgs = [i for j in imgs for i in j]
labels = [[i] * 4 for i in train_df.category_id.unique()]
labels = [i for j in labels for i in j]
for idx, img in enumerate(imgs):
    ax = fig.add_subplot(14, 4, idx + 1, xticks=[], yticks=[])
    im = Image.open("../input/train_images/" + img)
    plt.imshow(im)
    ax.set_title(f'Label: {labels[idx]}')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.shape

In [ ]:
test_df.shape

we can see that the category_id is the target variable.  Each category is an animal and the category 0 is "empty" that means no animal in the picture. 
I think useful variables are:
Date: we can define day, month, hour, season to understand when it is more luckly to see a particular.
Location: maybe animals are location specific. 
Seq_num_frames: to see if  it is more luckly to have an empty category when there is only one frame.


Let's do an assessment about what are the variables with highest correlation with the target. 


In [ ]:
trainset_c=train_df
labels = []
values = []
for col in trainset_c.columns:
    if col not in [ "category_id"] and trainset_c[col].dtype!='object':
        labels.append(col)
        values.append(np.corrcoef(trainset_c[col].values, trainset_c["category_id"].values)[0,1])
corr_df = pd.DataFrame({'columns_labels':labels, 'corr_values':values})
corr_df = corr_df.sort_values(by='corr_values')
 
corr_df = corr_df[(corr_df['corr_values']>0.20) | (corr_df['corr_values']<-0.20)]
ind = np.arange(corr_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(10,6))
rects = ax.barh(ind, np.array(corr_df.corr_values.values), color='gold')
ax.set_yticks(ind)
ax.set_yticklabels(corr_df.columns_labels.values, rotation='horizontal')
ax.set_xlabel("Correlation coefficient")
ax.set_title("Correlation coefficient of the variables")
plt.show()

In [ ]:
train_df['category_id'].loc[train_df['category_id']]
plt.figure(figsize=(12,10))
sns.violinplot(x='category_id', y='seq_num_frames', data=train_df)
plt.xlabel('category_id', fontsize=12)
plt.ylabel('seq_num_frames', fontsize=12)
plt.show()

we can see that if the catogory is empty the majority in this category has only one frame. The rest is quite well distributed. 


In [ ]:
from datetime import date, datetime
train_df['date_time'] = pd.to_datetime(train_df['date_captured'], errors='coerce')
train_df["year"] = train_df['date_time'].dt.year
train_df["month"] = train_df['date_time'].dt.month
train_df["day"] = train_df['date_time'].dt.day
train_df["hour"] = train_df['date_time'].dt.hour

train_df['season'] = train_df.month.map({1:4, 2:4, 3:1, 4:1, 5:1, 6:2, 7:2,8:2,9:3,10:3, 11:3,12:4})

#1= spring 
#2=summer
#3=autumn
#4=winter

In [ ]:
import matplotlib.ticker as ticker
train_df['category_id'].loc[train_df['category_id']]
plt.figure(figsize=(12,10))
ax= sns.violinplot(x='category_id', y='season', data=train_df)
plt.xlabel('category_id', fontsize=12)
plt.ylabel('season', fontsize=12)


#ax = sns.boxplot(data = np.random.rand(20,30))

ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
plt.show()

we can clearly see that the category_id=22 is appearing only  during summer and autumn

Rights_holder is also important because it could be that a specific photographer is specialised in some categories more than in others. Let's tranform this variable in 0 (Justin B.) or 1 (Erin..)

In [ ]:
train_df['Rights_holderN'] = train_df.rights_holder.map({'Justin Brown':0, 'Erin Boydston':1})

In [ ]:
train_df.head()

In [ ]:
import PIL
from PIL import Image
import os, sys


path = "../input/train_images/"
dirs = os.listdir( path )




def resizetrain():
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
          
            imResizeTrain = im.resize((32,32), Image.ANTIALIAS)
         #   imResize.save(z + ' resized.jpg', 'JPEG', quality=90)

resizetrain()

pathtest = "../input/test_images/"
dirs = os.listdir( path )

def resizetest():
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
          
            imResizeTest = im.resize((32,32), Image.ANTIALIAS)
         #   imResize.save(z + ' resized.jpg', 'JPEG', quality=90)

resizetest()

Scale images

In [ ]:
imResizeTrain   =  imResizeTrain.astype('float32')
imResizeTest =  imResizeTest.astype('float32')
imResizeTrain /= 255.
imResizeTest /= 255.
 

In [ ]:
generator_df= pd.DataFrame()
generator_df=  train_df[['category_id', 'id']]

In [ ]:
generator_df['category_id']=str('category_id')
generator_df.dtypes

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    shear_range=0.1,  # set range for random shear
    zoom_range=0.1,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=1/255.,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.1
)

def create_generator(subset):
    return datagen.flow_from_dataframe(
        generator_df, 
        imResizeTrain, 
        x_col='id',
        y_col=str('category_id'), 
        has_ext=True,  # If image extension is given in x_col
        target_size=(100, 100), 
        color_mode='rgb',
        class_mode='categorical', 
        batch_size=128, 
        shuffle=True, 
        seed=2018,
        subset=subset
    )

train_generator = create_generator('training')
val_generator = create_generator('validation')

In [ ]:
import keras
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import DenseNet121

reference:
https://www.kaggle.com/artgor/iwildcam-basic-eda
https://www.kaggle.com/xhlulu/keras-cnn-starter-petfinder/